## Load in Necessary Packages, etc.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys

sys.path.append('D:\substorm-detection\RNN')
sys.path.append('D:\substorm-detection')
sys.path.append('D:\substorm-detection\\baseline')
sys.path.append('D:\substorm-detection\\data')

import rnn_models
import utils
import utils_linear
import plot_utils
import linear_models

%load_ext autoreload
%autoreload 2

sns.set()

Using TensorFlow backend.


## Load in Data

In [2]:
folder = 'D:\\substorm-detection\\data'
data_fn = folder + '\\short_data.npz'
data_fn = folder + '\\2classes_data128_withsw.npz'
data = np.load(data_fn)
X = data['X']
y = data['y'][:, None]
SW = data['SW']
strength = data['strength']

print(np.shape(X))
print(np.shape(y))
print(np.shape(SW))
print(np.shape(strength))

(20446, 81, 128, 3)
(20446, 1)
(20457, 256, 6)
(20446,)


In [3]:
batch_size = 64

train_test_split = .2
train_val_split = .2
train, test = utils.split_data([X, SW, y, strength], train_test_split, random=False)
train, val = utils.split_data(train, train_val_split, random=True, batch_size=batch_size)
X_train, sw_train, y_train, strength_train = train
X_val, sw_val, y_val, strength_val = val
X_test, sw_test, y_test, strength_test = test

X_train, X_val, X_test = utils.rnn_format_x([X_train, X_val, X_test])
y_train, y_val, y_test = utils.rnn_format_y([y_train, y_val, y_test])

X_train, X_val, X_test = [X_train, sw_train], [X_val, sw_val], [X_test, sw_test]
y_train, y_val, y_test = [y_train, strength_train], [y_val, strength_val], [y_test, strength_test]

C:\Users\Alan\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Alan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Alan\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was 

In [8]:
params = {
    'batch_size': batch_size,
    'rnn_hidden_units': 64,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 5,
    'verbose': True,
    'time_output_weight': 1e6,
    'rnn_type': 'GRU',
    'output_type': 'time'
}

In [9]:
hist, model = rnn_models.train_functional_rnn_single(X_train, y_train, X_val, y_val, params)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (32, 256, 6)         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (32, 128, 243)       0                                            
__________________________________________________________________________________________________
gru_7 (GRU)                     (32, 256, 64)        13632       input_2[0][0]                    
__________________________________________________________________________________________________
gru_5 (GRU)                     (32, 128, 64)        59136       input_1[0][0]                    
__________________________________________________________________________________________________
gru_8 (GRU

In [10]:
hist, model = rnn_models.train_functional_rnn_combined(X_train, y_train, X_val, y_val, params)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (32, 256, 6)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (32, 128, 243)       0                                            
__________________________________________________________________________________________________
gru_11 (GRU)                    (32, 256, 64)        13632       input_4[0][0]                    
__________________________________________________________________________________________________
gru_9 (GRU)                     (32, 128, 64)        59136       input_3[0][0]                    
__________________________________________________________________________________________________
gru_12 (GR

### Training on only solar wind data

In [5]:
params = {
    'batch_size': batch_size,
    'rnn_hidden_units': 128,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 5,
    'verbose': True,
    'time_output_weight': 1,
    'rnn_type': 'GRU',
}

In [6]:
hist, model = rnn_models.train_basic_gru(X_train[1], y_train[0], X_val[1], y_val[0], params)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 13056 samples, validate on 3264 samples
Epoch 1/5
13056/13056 [==============================] - 241s 18ms/step - loss: 0.6399 - acc: 0.6225 - true_positive: 0.5591 - false_positive: 0.3194 - val_loss: 0.5893 - val_acc: 0.6887 - val_true_positive: 0.7364 - val_false_positive: 0.3544
Epoch 2/5
13056/13056 [==============================] - 239s 18ms/step - loss: 0.5782 - acc: 0.7027 - true_positive: 0.6615 - false_positive: 0.2616 - val_loss: 0.5665 - val_acc: 0.7188 - val_true_positive: 0.6660 - val_false_positive: 0.2327
Epoch 3/5
13056/13056 [==============================] - 250s 19ms/step - loss: 0.5704 - acc: 0.7142 - true_positive: 0.6576 - false_positive: 0.2330 - val_loss: 0.5731 - val_acc: 0.7062 - val_true_positive: 0.7124 - val_false_positive: 0.2990
Epoch 4/5
13056/13056 [==============================] - 239s 18ms/step - loss: 0.5660 - acc: 0.712

### Training on only magnetometer data

In [7]:
hist, model = rnn_models.train_basic_gru(X_train[0], y_train[0], X_val[0], y_val[0], params)

Train on 13056 samples, validate on 3264 samples
Epoch 1/5
13056/13056 [==============================] - 125s 10ms/step - loss: 0.5766 - acc: 0.7165 - true_positive: 0.6946 - false_positive: 0.2619 - val_loss: 0.5397 - val_acc: 0.7414 - val_true_positive: 0.7052 - val_false_positive: 0.2256
Epoch 2/5
13056/13056 [==============================] - 124s 9ms/step - loss: 0.5327 - acc: 0.7502 - true_positive: 0.7232 - false_positive: 0.2240 - val_loss: 0.5275 - val_acc: 0.7589 - val_true_positive: 0.7067 - val_false_positive: 0.1951
Epoch 3/5
13056/13056 [==============================] - 123s 9ms/step - loss: 0.5059 - acc: 0.7695 - true_positive: 0.7370 - false_positive: 0.2021 - val_loss: 0.5504 - val_acc: 0.7399 - val_true_positive: 0.7271 - val_false_positive: 0.2478
Epoch 4/5
13056/13056 [==============================] - 124s 9ms/step - loss: 0.4956 - acc: 0.7726 - true_positive: 0.7390 - false_positive: 0.1969 - val_loss: 0.5234 - val_acc: 0.7592 - val_true_positive: 0.7491 - val_f

#### Some Hyperparameter Tuning

In [13]:
params = {
    'batch_size': batch_size,
    'rnn_hidden_units': 32,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 10,
    'verbose': True,
    'time_output_weight': 1e6,
    'rnn_type': 'GRU',
    'output_type': 'time'
}

In [14]:
hist, model = rnn_models.train_functional_rnn_single(X_train, y_train, X_val, y_val, params)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (32, 256, 6)         0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (32, 128, 243)       0                                            
__________________________________________________________________________________________________
gru_19 (GRU)                    (32, 256, 32)        3744        input_8[0][0]                    
__________________________________________________________________________________________________
gru_17 (GRU)                    (32, 128, 32)        26496       input_7[0][0]                    
__________________________________________________________________________________________________
gru_20 (GR

In [15]:
params = {
    'batch_size': batch_size,
    'rnn_hidden_units': 16,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 10,
    'verbose': True,
    'time_output_weight': 1e6,
    'rnn_type': 'GRU',
    'output_type': 'time'
}

In [16]:
hist, model = rnn_models.train_functional_rnn_single(X_train, y_train, X_val, y_val, params)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (32, 256, 6)         0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (32, 128, 243)       0                                            
__________________________________________________________________________________________________
gru_23 (GRU)                    (32, 256, 16)        1104        input_10[0][0]                   
__________________________________________________________________________________________________
gru_21 (GRU)                    (32, 128, 16)        12480       input_9[0][0]                    
__________________________________________________________________________________________________
gru_24 (GR

In [19]:
params = {
    'batch_size': batch_size,
    'rnn_hidden_units': 64,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 10,
    'verbose': True,
    'time_output_weight': 1e6,
    'rnn_type': 'GRU',
    'output_type': 'time',
    'dropout_rate': 0.2
}

In [20]:
hist, model = rnn_models.train_functional_rnn_single(X_train, y_train, X_val, y_val, params)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (32, 256, 6)         0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (32, 128, 243)       0                                            
__________________________________________________________________________________________________
gru_31 (GRU)                    (32, 256, 64)        13632       input_14[0][0]                   
__________________________________________________________________________________________________
gru_29 (GRU)                    (32, 128, 64)        59136       input_13[0][0]              